In [ ]:
import re

class Component:
    def __init__(self, name, comp_type, position):
        self.name = name
        self.type = comp_type
        self.position = position  # (x, y)
        
    def __repr__(self):
        return f"{self.type}({self.name}) at {self.position}"


class Pad(Component):
    def __init__(self, name, position, radius):
        super().__init__(name, 'pad', position)
        self.radius = radius

    def __repr__(self):
        return f"Pad({self.name}, pos={self.position}, r={self.radius})"


class RectPad(Component):
    def __init__(self, name, position, side_length, angle):
        super().__init__(name, 'rect_pad', position)
        self.side_length = side_length
        self.angle = angle  # degrees

    def __repr__(self):
        return f"RectPad({self.name}, pos={self.position}, side={self.side_length}, angle={self.angle})"

class good_board:
    supported_list = {'wire', 'pad', 'rect_pad'}

    def __init__(self, file):
        self.components = {}  # name -> Component instance
        self.supported_list_activated = set()
        self.parse_file(file)

    def parse_file(self, file):
        with open(file, "r") as fo:
            for line in fo:
                line = line.strip()
                if not line:
                    continue
                try:
                    name, rest = line.split(":")
                    name = name.strip()
                    type_and_params = rest.split(",", 1)
                    comp_type = type_and_params[0].strip()
    
                    if comp_type not in self.supported_list:
                        print(f"Unsupported component type: {comp_type}")
                        continue
    
                    self.supported_list_activated.add(comp_type)
    
                    if comp_type == 'pad':
                        # 使用正規表達式提取位置與 radius
                        match = re.match(r"\s*\(([^)]+)\)\s*,\s*(.+)", type_and_params[1])
                        if not match:
                            raise ValueError("Invalid pad format")
                        pos_str, radius_str = match.groups()
                        x, y = map(float, pos_str.split(","))
                        radius = float(radius_str)
                        self.components[name] = Pad(name, (x, y), radius)
    
                    elif comp_type == 'rect_pad':
                        match = re.match(r"\s*\(([^)]+)\)\s*,\s*([-\d.]+)\s*,\s*([-\d.]+)", type_and_params[1])
                        if not match:
                            raise ValueError("Invalid rect_pad format")
                        pos_str, side_str, angle_str = match.groups()
                        x, y = map(float, pos_str.split(","))
                        side = float(side_str)
                        angle = float(angle_str)
                        self.components[name] = RectPad(name, (x, y), side, angle)
    
                except Exception as e:
                    print(f"Failed to parse line: '{line}', error: {e}")
    
    
        def __str__(self):
            return "\n".join(str(c) for c in self.components.values())

In [47]:
# 建立 good_board 實例並讀取 foo.txt
board = good_board("foo.txt")

# 印出所有元件
print("=== Components ===")
print(board)

# 顯示每個元件詳細資訊
for name, comp in board.components.items():
    print(f"\n{name} details:")
    print(f"  Type: {comp.type}")
    print(f"  Position: {comp.position}")
    if comp.type == 'pad':
        print(f"  Radius: {comp.radius}")
    elif comp.type == 'rect_pad':
        print(f"  Side Length: {comp.side_length}")
        print(f"  Angle: {comp.angle}")

=== Components ===

pad1 details:
  Type: pad
  Position: (10.0, 10.0)
  Radius: 2.0

rect_pad1 details:
  Type: rect_pad
  Position: (-10.0, -10.0)
  Side Length: 5.0
  Angle: 45.0
